In [1]:
# coding: utf-8

# In[1]:




# In[1]:
get_ipython().magic(u'matplotlib inline')

# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()


# In[2]:

'3.5.2'

In [2]:
# In[2]:


try:
    import scrapy
except:
    get_ipython().system('python3 -m pip install scrapy')
    import scrapy
from scrapy.crawler import CrawlerProcess
class DmozItem(scrapy.Item):
    date=scrapy.Field()
    title = scrapy.Field()
    studio=scrapy.Field()


# In[3]:

In [3]:
# In[3]:


import json

class JsonWriterPipeline2(object):

    def open_spider(self, spider):
        self.file = open('quoterest2.jl', 'w+')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


# In[4]:

In [4]:
# In[4]:


class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoterest.jl', 'w+')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


# In[5]:

In [5]:
# In[5]:


class JsonWriterPipeline3(object):

    def open_spider(self, spider):
        self.file = open('quoterest3.jl', 'w+')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [6]:
# In[6]:


import logging

class QuotesSpiderVan(scrapy.Spider):
    name = "quotesVan"
    start_urls = [
        'http://vfxvancouver.com/page/1/?s']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline3': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoterest3.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for sel in response.xpath(".//article[contains(@id, 'post')]"):
            print( sel)
            item = DmozItem()
            item["date"] = sel.xpath('''.//div[@class='entry-meta']/data//text()''').extract()       
            item["title"]= sel.xpath(".//h1[@class='entry-title']/a/text()").extract()
            item["studio"]= sel.xpath(".//a[@class='author-link']/@href").extract()

            print( item)
            if len(item["date"])==0:
                item["date"].append("jan 1, 2018")
            if len(item["title"])==0:
                item["title"].append("none")
            if len(item["studio"])==0:
                item["studio"].append("none")      
            dat=item["date"][0]
            ti=item["title"][0]
            st=item["studio"][0]

            
            yield {'date':dat,'title':ti,'studio':st}
        url_older=response.xpath('.//a[@class="next page-numbers"]//@href').extract()
        if (len(url_older))>0:
            print (url_older)
            abs_url=url_older[0]
            print (abs_url )
            yield scrapy.Request(abs_url) 
        else:
            print("Last element")
        return

In [7]:
# In[7]:




class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://www.torontovfxjobs.com/jobs/']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoterest.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for sel in response.xpath("//header[@class='entry-header']"):
            print( sel)
            item = DmozItem()
            item["date"] = sel.xpath('''.//div[@class='meta-below-title']//time[@class="dt-published published entry-date"]//text()''').extract()       
            item["title"]= sel.xpath(".//a[@class='entry-title-passthrough']/text()").extract()
            item["studio"]= sel.xpath(".//a[@class='entry-title-passthrough']/@href").extract()

            print( item)
            if len(item["date"])==0:
                item["date"].append("jan 1, 2018")
            if len(item["title"])==0:
                item["title"].append("none")
            if len(item["studio"])==0:
                item["studio"].append("none")      
            dat=item["date"][0]
            ti=item["title"][0]
            st=item["studio"][0]

            
            yield {'date':dat,'title':ti,'studio':st}
        url_older=response.xpath('//div[@class="older"]//@href').extract()
        if (len(url_older))>0:
            print (url_older)
            abs_url=response.urljoin(url_older[0].split("/")[-1])
            print (abs_url )
            yield scrapy.Request(abs_url) 
        else:
            print("Last element")
        return

In [8]:
# In[8]:


# In[6]:


class JobSpider(scrapy.Spider):
    name = 'job'
    allowed_domains = ['vfx-montreal.com/jobs/']
    start_urls = ['http://vfx-montreal.com/jobs/']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline2': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoterest2.json'  }                      # Used for pipe
    def parse(self, response):
        blocks=response.xpath('//div[@class="mk-text-block  "]')
        i=0
        for block in blocks:
            titleJob=block.xpath ('.//div[@class="vfx-job-list-item-title"]/a/text()').extract_first()
            dateJob=block.xpath ('.//div[@class="vfx-job-list-item-end-date"]/text()').extract_first()
            studioJob=block.xpath ('.//div[@class="vfx-job-list-item-studio-name"]/a/@href').extract_first()
            i=i+1
            dateJobForm=dateJob.replace("Published: ",'')
            try:
                print (titleJob)
                print (dateJob)
                print (studioJob)
                yield {'date':dateJobForm,'title':titleJob,'studio':studioJob}

            except:
                print('i')

In [9]:
# In[15]:


import pandas as pd
import matplotlib as mp
import os
print("1:Read local data"+"\n"+"2:Read online data")
nb= input("Choose a number")
if nb=="2":
    file='quoterest.json'
    raw = open(file, "w+")
    raw.seek(0)                       
    raw.truncate()
    raw.close()
    file= 'quoterest2.json'
    raw = open(file, "w+")
    raw.seek(0)                       
    raw.truncate()
    raw.close()
    file= 'quoterest3.json'
    raw = open(file, "w+")
    raw.seek(0)                       
    raw.truncate()
    raw.close()


process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.crawl(JobSpider)
process.crawl(QuotesSpiderVan)
process.start()

1:Read local data
2:Read online data
Choose a number2


0

0

0

0

...

<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3798" class="row post-'>
{'date': ['July 29, 2015'],
 'studio': ['http://vfxvancouver.com/author/waterproof-studios/'],
 'title': ['Senior FX Artists']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3796" class="row post-'>
{'date': ['July 29, 2015'],
 'studio': ['http://vfxvancouver.com/author/waterproof-studios/'],
 'title': ['Modelers and Surfacing Artists']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3794" class="row post-'>
{'date': ['July 29, 2015'],
 'studio': ['http://vfxvancouver.com/author/waterproof-studios/'],
 'title': ['Junior and Senior Animators']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3792" class="row post-'>
{'date': ['July 29, 2015'],
 'studio': ['http://vfxvancouver.com/author/cosa/'],
 'title': ['VFX Coordinator/PA']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="pos

<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3577" class="row post-'>
{'date': ['June 17, 2015'],
 'studio': ['http://vfxvancouver.com/author/cosa/'],
 'title': ['Visual Effects Artists/Leads for TV Series – Mid to Senior']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3575" class="row post-'>
{'date': ['June 17, 2015'],
 'studio': ['http://vfxvancouver.com/author/goldtooth/'],
 'title': ['Compositor']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3847" class="row post-'>
{'date': ['June 15, 2015'],
 'studio': ['http://vfxvancouver.com/author/ria/'],
 'title': ['Advertising Artist']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3568" class="row post-'>
{'date': ['June 12, 2015'],
 'studio': ['http://vfxvancouver.com/author/fusefx/'],
 'title': ['Compositing Lead']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3565" class="row post-'>

 'title': ['SENIOR COMPOSITORS']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3347" class="row post-'>
{'date': ['April 2, 2015'],
 'studio': ['http://vfxvancouver.com/author/waterproof-studios/'],
 'title': ['Junior and Senior Animators']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3343" class="row post-'>
{'date': ['March 27, 2015'],
 'studio': ['http://vfxvancouver.com/author/go2productions/'],
 'title': ['3D Generalist']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3341" class="row post-'>
{'date': ['March 20, 2015'],
 'studio': ['http://vfxvancouver.com/author/atmosphere-vfx/'],
 'title': ['I/O Editor']}
['http://vfxvancouver.com/page/105/?s']
http://vfxvancouver.com/page/105/?s
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3339" class="row post-'>
{'date': ['March 19, 2015'],
 'studio': ['http://vfxvancouver.com/author/atmosphere-vfx/'],
 'title': ['Senio

<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3030" class="row post-'>
{'date': ['January 15, 2015'],
 'studio': ['http://vfxvancouver.com/author/adam/'],
 'title': ['VFX Animators']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3028" class="row post-'>
{'date': ['January 15, 2015'],
 'studio': ['http://vfxvancouver.com/author/adam/'],
 'title': ['3D GENERALISTS']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3025" class="row post-'>
{'date': ['January 15, 2015'],
 'studio': ['http://vfxvancouver.com/author/fun-square/'],
 'title': ['VFX Compositors']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-3014" class="row post-'>
{'date': ['January 9, 2015'],
 'studio': ['http://vfxvancouver.com/author/waterproof-studios/'],
 'title': ['Jr. and Sr. Animators']}
['http://vfxvancouver.com/page/107/?s']
http://vfxvancouver.com/page/107/?s
<Selector xpath=".//article[contains(@

<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-2905" class="row post-'>
{'date': ['December 6, 2014'],
 'studio': ['http://vfxvancouver.com/author/hydraulxvfx/'],
 'title': ['2D Senior/Mid Level Compositor for Immediate hire']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-2898" class="row post-'>
{'date': ['December 3, 2014'],
 'studio': ['http://vfxvancouver.com/author/waterproof-studios/'],
 'title': ['Lead Rigging TD']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-2892" class="row post-'>
{'date': ['December 2, 2014'],
 'studio': ['http://vfxvancouver.com/author/the-embassy/'],
 'title': ['Lighting/Shading TD']}
<Selector xpath=".//article[contains(@id, 'post')]" data='<article id="post-2888" class="row post-'>
{'date': ['December 2, 2014'],
 'studio': ['http://vfxvancouver.com/author/the-embassy/'],
 'title': ['Animation Generalist']}
<Selector xpath=".//article[contains(@id, 'post')]" data='

In [10]:
# In[13]:


dfjson = pd.read_json('quoterest.json')

print(dfjson)


dfjson2= pd.read_json('quoterest2.json')

print(dfjson2)
dfjson3= pd.read_json('quoterest3.json')

print(dfjson3)

          date                                             studio  \
0   2018-07-10  https://intelligent-creatures.breezy.hr/p/bb7f...   
1   2018-07-05  https://intelligent-creatures.breezy.hr/p/061e...   
2   2018-07-05  https://intelligent-creatures.breezy.hr/p/bc12...   
3   2018-07-05  https://intelligent-creatures.breezy.hr/p/9b94...   
4   2018-07-05  https://intelligent-creatures.breezy.hr/p/bf39...   
5   2018-06-26  https://www.industrialbrothers.com/jobs/cg-vfx...   
6   2018-06-26  https://www.industrialbrothers.com/jobs/cg-sup...   
7   2018-06-26  https://www.industrialbrothers.com/jobs/lead-c...   
8   2018-06-26  https://www.industrialbrothers.com/jobs/techni...   
9   2018-06-26  https://www.industrialbrothers.com/jobs/cg-rig...   
10  2018-06-21  https://rocketsciencevfx.applytojob.com/apply/...   
11  2018-06-21  https://rocketsciencevfx.applytojob.com/apply/...   
12  2018-06-21  https://rocketsciencevfx.applytojob.com/apply/...   
13  2018-06-21  https://rocketscie

           date                                             studio  \
0    2018-07-16   http://vfxvancouver.com/author/nomadic-pictures/   
1    2018-07-16              http://vfxvancouver.com/author/siyun/   
2    2018-07-16  http://vfxvancouver.com/author/sony-pictures-i...   
3    2018-07-12             http://vfxvancouver.com/author/bardel/   
4    2018-07-12  http://vfxvancouver.com/author/sony-pictures-i...   
5    2018-07-11     http://vfxvancouver.com/author/atmosphere-vfx/   
6    2018-07-11     http://vfxvancouver.com/author/atmosphere-vfx/   
7    2018-07-11     http://vfxvancouver.com/author/atmosphere-vfx/   
8    2018-07-10           http://vfxvancouver.com/author/vitality/   
9    2018-07-10       http://vfxvancouver.com/author/animal-logic/   
10   2018-07-10            http://vfxvancouver.com/author/goh-van/   
11   2018-07-10            http://vfxvancouver.com/author/goh-van/   
12   2018-07-06       http://vfxvancouver.com/author/iconcreative/   
13   2018-07-06     

In [11]:
# In[16]:



if nb=="1":
    fLoc=open('quoterest.json','r')
    fLoc2=open('quoterest2.json',"r")
    fLoc3=open('quoterest3.json',"r")
    fSize=os.stat('quoterest.json').st_size
    fSize2=os.stat('quoterest2.json').st_size
    fSize3=os.stat('quoterest3.json').st_size
    if (fSize!=0)&(fSize2!=0)&(fSize3!=0):
        dfjson = pd.read_json(fLoc)
        dfjsonConv=pd.to_datetime(dfjson['date'],infer_datetime_format=True)
        print(dfjsonConv)


        dfjson2= pd.read_json(fLoc2)

        print(dfjson2)
        dfjson3= pd.read_json(fLoc3)

        print(dfjson3)

        frames=[dfjson,dfjson2,dfjson3]
        dfConcat=pd.concat(frames)
        dfConcat.index=dfConcat['date']
        dfConcatRes=dfConcat.sort_index()

        print(dfConcatRes)
        dfNewComp=dfConcatRes[dfConcatRes["title"].str.contains("Compositor|compsitor|Comper|COMPOSITOR")]
        print(dfNewComp)
        dfNewComp.to_pickle("comper.pkl")
        dfNewGen=dfConcatRes[dfConcatRes['title'].str.contains("Generalist|GENERALIST")]
        print(dfNewGen)
        dfNewSup=dfConcatRes[dfConcatRes['title'].str.contains("Supervisor|SUPERVISOR")]
        print(dfNewSup)
        dfNewMan=dfConcatRes[dfConcatRes['title'].str.contains("Manager|Producer")]
        print(dfNewMan)
        dfNewTra=dfConcatRes[dfConcatRes['title'].str.contains("Tracking|Camera|Tracker|Matcmove|Matcmover")]
        print(dfNewTra)
        dfNewPai=dfConcatRes[dfConcatRes['title'].str.contains("PAINTER|Painter|painter|Matte|MATTE")]
        print(dfNewPai)
        dfNewAni=dfConcatRes[dfConcatRes['title'].str.contains("Animator|Animation|Rigger|RIGGER|Rig")]
        print(dfNewAni)
        dfNewMod=dfConcatRes[dfConcatRes['title'].str.contains("Modeller|MODELLER|Modeler")]
        print(dfNewMod)
        dfNewComp.groupby(dfNewComp["date"].dt.month).count().plot(kind="bar")

else:
    print("fileEmpty")

fileEmpty


In [12]:
# In[18]:


dfNewComp.groupby(dfNewComp["date"].dt.year).count().plot(kind="bar")


# In[ ]:


dfConcatRes.groupby(dfConcatRes["date"].dt.year).count().plot(kind="bar")

NameError: name 'dfNewComp' is not defined